In [1]:
# Importações
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

In [2]:
# Definição de funções necessárias

def get_noticias(url, keywords):
    """
    Função que coleta notícias de uma página específica baseada em palavras-chave nos links.
    Também coleta a data de publicação, se disponível.
    """
    noticias = []
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.encoding = response.apparent_encoding  # Ajuste para decodificação correta
        soup = BeautifulSoup(response.content, "html.parser")

        # Encontrar todos os links na página
        links = soup.find_all('a', href=True)

        # Filtrar e armazenar apenas os links que contêm as palavras-chave
        for link_tag in links:
            link = link_tag['href']
            title = link_tag.get_text(strip=True)
            
            # Verificar se a palavra-chave está no link ou no título
            if any(keyword in title.lower() for keyword in keywords):
                # Se o link não é absoluto, fazê-lo absoluto
                if not link.startswith('http'):
                    link = urljoin(url, link)

                # Tentar capturar a data de publicação da notícia
                date = 'Data não disponível'
                try:
                    date_tag = soup.find('time')  # Se houver uma tag <time> na página
                    if date_tag and date_tag.has_attr('datetime'):
                        date = date_tag['datetime']
                    else:
                        # Tentativa alternativa, por exemplo, se a data estiver em um <span> com uma classe específica
                        date_tag = soup.find('span', class_='data-publicacao')
                        date = date_tag.get_text(strip=True) if date_tag else 'Data não disponível'
                except Exception:
                    pass

                # Adicionar a notícia à lista, incluindo a data
                if title:
                    noticias.append({"title": title, "link": link, "date": date})
    
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
    
    return noticias

def crawl_site(url, keywords, depth=3):
    """
    Função recursiva para percorrer todo o site, coletando notícias relevantes.
    Agora coleta a data da notícia.
    """
    visited_links = set()
    noticias = []

    def _crawl(url, current_depth):
        # Verificar se o link já foi visitado ou se a profundidade máxima foi atingida
        if current_depth == 0 or url in visited_links:
            return
        
        visited_links.add(url)  # Marcar o link como visitado
        noticias_atual = get_noticias(url, keywords)

        # Remover links duplicados antes de adicionar à lista de notícias
        for noticia in noticias_atual:
            if noticia["link"] not in {n["link"] for n in noticias}:
                noticias.append(noticia)

        try:
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            response.encoding = response.apparent_encoding
            soup = BeautifulSoup(response.content, "html.parser")

            # Encontrar todos os links na página e seguir
            links = soup.find_all('a', href=True)

            for link_tag in links:
                link = link_tag['href']
                if not link.startswith('http'):
                    link = urljoin(url, link)

                # Verificar se o link é interno e não foi visitado
                if url in link and link not in visited_links:
                    _crawl(link, current_depth - 1)

        except requests.exceptions.RequestException as e:
            print(f"Erro ao acessar {url}: {e}")

    _crawl(url, depth)
    return noticias

def get_text(url):
    '''
    Função 
    '''
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.content, 'html.parser')

        for script in soup(['script', 'style']):
            script.extract()

        text = soup.get_text()

        lines = (line.strip() for line in text.splitlines())

        chunks = (phrase.strip() for line in lines for phrase in line.split())

        text = ' '.join(chunk for chunk in chunks if chunk)

        return text
    
    except Exception as e:
        print(f"Erro ao obter texto de {url}: {e}")
        return ""

def coluna_texto(df):
    if 'link' in df:
        df['corpo_texto'] = df['link'].apply(get_text)
    else:
        pass
    
def extrair_para_df(data_dict, keywords):
    base_url = data_dict['base_url']
    depth = data_dict['depth']
    
    # Coletar notícias do site inteiro, seguindo links internos
    df_nome = pd.DataFrame(crawl_site(base_url, keywords, depth=depth))
    
    # Remover títulos duplicados
    df_nome = df_nome.drop_duplicates(subset='title')
    
    # Adicionar a coluna 'full_text' com o texto completo extraído de cada link
    coluna_texto(df_nome)
    
    return df_nome

In [3]:
# Palavras-chave a serem buscadas
keywords = ["vacina", "vacinação", "vacinacao", "covid", "pandemia", "corona", "coronavírus", "coronavirus", "quarentena", "sars-cov-2"]

In [4]:
# Definição dos dados a serem extraídos
true_data = {
    'g1 saude':{
        'base_url':'https://g1.globo.com/saude/',
        'depth':4
    },
    'gov saude':{
        'base_url':'https://www.gov.br/saude/pt-br',
        'depth':3
    }
}

fake_data = {
    'fato ou fake g1':{
        'base_url':'https://g1.globo.com/fato-ou-fake/',
        'depth':6
    },
    'aos fatos':{
        'base_url':'https://www.aosfatos.org',
        'depth':3
    },
    'gov fake':{
        'base_url':'https://www.saude.gov.br/fakenews',
        'depth':3
    },
    'boatos':{
        'base_url':'https://www.boatos.org',
        'depth':3
    },
    'mpv':{
        'base_url':'https://medicospelavidacovid19.com.br/',
        'depth':5
    }    
}

In [5]:
# Extrai dados da internet e cria uma lista com os dfs (falsos e verdadeiros)
fake_dfs = []
for data_dict in fake_data.values():
    fake_dfs.append(extrair_para_df(data_dict, keywords))
    
true_dfs = []
for data_dict in true_data.values():
    true_dfs.append(extrair_para_df(data_dict, keywords))

/home/alice.ferreira/anaconda3/lib/python3.12/html/parser.py:171: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


Erro ao obter texto de mailto:contato@medicospelavidacovid19.com.br: No connection adapters were found for 'mailto:contato@medicospelavidacovid19.com.br'


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Erro ao obter texto de https://www.gov.br/saude/pt-br/assuntos/pcdt/arquivos/2022/portal-portaria-gm-2-994-lciam-e-protocolo-de-sca-1.pdf: The markup you provided was rejected by the parser. Trying a different parser or a different encoding may help.

Original exception(s) from parser:
 AssertionError: expected name token at '<![���aFe\x0b�\x0b�g�\x1d\x15�W�'


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Erro ao obter texto de http://%20https//www.gov.br/saude/pt-br/coronavirus/vacinas/plano-nacional-de-operacionalizacao-da-vacina-contra-a-covid-19: HTTPConnectionPool(host='%20https', port=80): Max retries exceeded with url: /www.gov.br/saude/pt-br/coronavirus/vacinas/plano-nacional-de-operacionalizacao-da-vacina-contra-a-covid-19 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7fdac5f81c70>: Failed to resolve '%20https' ([Errno -2] Name or service not known)"))
Erro ao obter texto de https://registra-rh-covid19.saude.gov.br/: HTTPSConnectionPool(host='registra-rh-covid19.saude.gov.br', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fdac3edbe30>: Failed to resolve 'registra-rh-covid19.saude.gov.br' ([Errno -2] Name or service not known)"))
Erro ao obter texto de https:// https//antigo.saude.gov.br/images/pdf/2020/marco/04/Calendario-Vacinao-2020-Crian--a.pdf: HTTPSConnectionPool

In [6]:
# Devido à estrutura desse site, precisamos mudar a sintaxe para o www.e-farsas.com
base_url = "https://www.e-farsas.com/secoes/verdadeiro-2/page"

# Coletar notícias de todas as páginas do site, seguindo links internos
df_efarsa_verdadeiro = []
for i in range(1, 159):  # Número de páginas a ser percorrido
    page_url = f"{base_url}{i}"
    df_efarsa_verdadeiro.append(extrair_para_df({'base_url':page_url,'depth':3}, keywords))

df_efarsa_verdadeiro =  pd.concat(df_efarsa_verdadeiro, ignore_index=True)
true_dfs.append(df_efarsa_verdadeiro)


base_url = "https://www.e-farsas.com/secoes/falso-2/page"

df_efarsa_falso = []
for i in range(1, 159):
    page_url = f"{base_url}{i}"
    df_efarsa_falso.append(extrair_para_df({'base_url':page_url,'depth':3}, keywords))

df_efarsa_falso =  pd.concat(df_efarsa_falso, ignore_index=True)
fake_dfs.append(df_efarsa_falso)

In [7]:
# Junta os dfs fakes e salva
df_combinedfake = pd.concat(fake_dfs, ignore_index=True)
df_combinedfake.to_csv("dados/df_combinedfake.csv", index=False, encoding='utf-8')
display(df_combinedfake)

# Junta os verdadeiros e salva
df_combinedtrue = pd.concat(true_dfs, ignore_index=True)
df_combinedtrue.to_csv("dados/df_combinedtrue.csv", index=False, encoding='utf-8')
display(df_combinedtrue)

# Junta todos os dfs extraídos em um só e salva como .csv
df_combinedextracted = pd.concat([df_combinedfake, df_combinedtrue])
df_combinedextracted.to_csv("dados/df_combinedextracted.csv", index=False, encoding='utf-8')
display(df_combinedextracted)

,title,link,date,corpo_texto
0,Coronavirus,https://g1.globo.com/saude/coronavirus/,Data não disponível,g1 Coronavirus: Tudo sobre o COVID-19 Coronaví...
1,lucraram com desinformação sobre a pandemia,https://www.aosfatos.org/noticias/como-sete-si...,Data não disponível,Como sete sites lucraram com anúncios no Googl...
2,Covid-19,https://www.gov.br/saude/pt-br/assuntos/covid-19,Data não disponível,Covid-19 — Ministério da Saúde Ir para o Conte...
3,Vacinação,https://www.gov.br/saude/pt-br/vacinacao,Data não disponível,Vacinação — Ministério da Saúde Ir para o Cont...
4,Calendário de Vacinação,https://www.gov.br/saude/pt-br/vacinacao/calen...,Data não disponível,Calendário de Vacinação — Ministério da Saúde ...
...,...,...,...,...
242,Uma jovem de 17 anos morreu por causa da vacin...,http://www.e-farsas.com/uma-jovem-de-17-anos-m...,Data não disponível,Uma jovem de 17 anos morreu por causa da vacin...
243,A vacina contra a febre amarela é um veneno mo...,http://www.e-farsas.com/vacina-contra-febre-am...,Data não disponível,A vacina contra a febre amarela é um veneno mo...
244,Vacina contra a rubéola foi a causa da microce...,http://www.e-farsas.com/vacina-contra-a-rubeol...,Data não disponível,Vacina contra a rubéola foi a causa da microce...
245,Vacina anticâncer rins e pele! Noticia boa (se...,http://www.e-farsas.com/vacina-anticancer-rins...,Data não disponível,Vacina anticâncer rins e pele! Noticia boa (se...


,title,link,date,corpo_texto
0,Coronavirus,https://g1.globo.com/saude/coronavirus/,Data não disponível,g1 Coronavirus: Tudo sobre o COVID-19 Coronaví...
1,Trump escolhe ativista antivacina para chefiar...,https://g1.globo.com/mundo/eleicoes-nos-eua/20...,2024-11-14T18:46:07.420Z,"Antivacina, Robert F. Kennedy Jr. é escolhido ..."
2,Covid-19 não foi 'gripezinha' e negacionismo p...,https://g1.globo.com/politica/noticia/2024/11/...,Data não disponível,Covid-19 não foi 'gripezinha' e negacionismo p...
3,Bem-Estar #272: Aumento da coqueluche e mudanç...,https://g1.globo.com/bemestar/podcast/noticia/...,Data não disponível,Bem-Estar #272: Aumento da coqueluche e mudanç...
4,Saúde compra 69 milhões de doses de vacina con...,https://g1.globo.com/saude/noticia/2024/11/05/...,Data não disponível,Saúde compra 69 milhões de doses de vacina con...
...,...,...,...,...
255,Governo do Maranhão vem fornecendo kits com cl...,http://www.e-farsas.com/governo-do-maranhao-ve...,Data não disponível,Governo do Maranhão vem fornecendo kits com cl...
256,“Kit COVID-19” para combater o novo coronavíru...,http://www.e-farsas.com/kit-covid-19-para-comb...,Data não disponível,"""Kit COVID-19"" para combater o novo coronavíru..."
257,"Autoridades, parem de distorcer fatos sobre a ...",http://www.e-farsas.com/autoridades-parem-de-d...,Data não disponível,"Autoridades, parem de distorcer fatos sobre a ..."
258,O livro “A realidade de Madhu” de 2013 previu ...,http://www.e-farsas.com/o-livro-a-realidade-de...,Data não disponível,O livro “A realidade de Madhu” de 2013 previu ...


,title,link,date,corpo_texto
0,Coronavirus,https://g1.globo.com/saude/coronavirus/,Data não disponível,g1 Coronavirus: Tudo sobre o COVID-19 Coronaví...
1,lucraram com desinformação sobre a pandemia,https://www.aosfatos.org/noticias/como-sete-si...,Data não disponível,Como sete sites lucraram com anúncios no Googl...
2,Covid-19,https://www.gov.br/saude/pt-br/assuntos/covid-19,Data não disponível,Covid-19 — Ministério da Saúde Ir para o Conte...
3,Vacinação,https://www.gov.br/saude/pt-br/vacinacao,Data não disponível,Vacinação — Ministério da Saúde Ir para o Cont...
4,Calendário de Vacinação,https://www.gov.br/saude/pt-br/vacinacao/calen...,Data não disponível,Calendário de Vacinação — Ministério da Saúde ...
...,...,...,...,...
255,Governo do Maranhão vem fornecendo kits com cl...,http://www.e-farsas.com/governo-do-maranhao-ve...,Data não disponível,Governo do Maranhão vem fornecendo kits com cl...
256,“Kit COVID-19” para combater o novo coronavíru...,http://www.e-farsas.com/kit-covid-19-para-comb...,Data não disponível,"""Kit COVID-19"" para combater o novo coronavíru..."
257,"Autoridades, parem de distorcer fatos sobre a ...",http://www.e-farsas.com/autoridades-parem-de-d...,Data não disponível,"Autoridades, parem de distorcer fatos sobre a ..."
258,O livro “A realidade de Madhu” de 2013 previu ...,http://www.e-farsas.com/o-livro-a-realidade-de...,Data não disponível,O livro “A realidade de Madhu” de 2013 previu ...
